In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()*_input.size(0) / (counter * train_loader.batch_size)))

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 5

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(4):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
32150,"[580.6542358398438, 1382.8809814453125]","[580.6947631835938, 1382.5338134765625]","[580.6333618164062, 1382.1611328125]","[580.5606079101562, 1381.699462890625]","[580.5039672851562, 1381.291015625]","[580.4371948242188, 1380.88671875]","[580.3681640625, 1380.478759765625]","[580.297119140625, 1380.06787109375]","[580.2244873046875, 1379.65234375]","[580.1507568359375, 1379.23193359375]",...,"[579.3914794921875, 1374.2637939453125]","[579.3369140625, 1373.7850341796875]","[579.286376953125, 1373.302734375]","[579.239990234375, 1372.8172607421875]","[579.1981811523438, 1372.3291015625]","[579.1611938476562, 1371.83837890625]","[579.1292114257812, 1371.3455810546875]","[579.1024780273438, 1370.85107421875]","[579.0811767578125, 1370.3551025390625]","[579.0654907226562, 1369.8580322265625]"
6629,"[2147.9453125, 698.1753540039062]","[2146.484375, 696.8275756835938]","[2145.405029296875, 695.7474365234375]","[2144.47412109375, 694.8160400390625]","[2143.590576171875, 693.9466552734375]","[2142.716064453125, 693.0996704101562]","[2141.834716796875, 692.2567138671875]","[2140.939453125, 691.4085693359375]","[2140.026123046875, 690.550537109375]","[2139.092529296875, 689.6802978515625]",...,"[2127.900390625, 679.6448974609375]","[2126.892822265625, 678.7667846679688]","[2125.906982421875, 677.9067993164062]","[2124.946533203125, 677.0657348632812]","[2124.01513671875, 676.2437133789062]","[2123.11572265625, 675.43994140625]","[2122.250732421875, 674.6531372070312]","[2121.421875, 673.8814086914062]","[2120.629638671875, 673.1224975585938]","[2119.874755859375, 672.3740844726562]"
36283,"[414.6539001464844, 1526.989013671875]","[415.17926025390625, 1528.2720947265625]","[415.21002197265625, 1529.1878662109375]","[415.3946838378906, 1530.1072998046875]","[415.525634765625, 1531.0667724609375]","[415.66644287109375, 1532.026611328125]","[415.80560302734375, 1532.9847412109375]","[415.945556640625, 1533.937744140625]","[416.08551025390625, 1534.88427734375]","[416.2249450683594, 1535.8233642578125]",...,"[417.54736328125, 1545.771728515625]","[417.6380920410156, 1546.6654052734375]","[417.72308349609375, 1547.560546875]","[417.80230712890625, 1548.456787109375]","[417.8756103515625, 1549.353759765625]","[417.9429931640625, 1550.2503662109375]","[418.0043029785156, 1551.1453857421875]","[418.0595397949219, 1552.036865234375]","[418.108642578125, 1552.9224853515625]","[418.15167236328125, 1553.79931640625]"
29285,"[747.484375, 1374.366455078125]","[747.3018798828125, 1375.15478515625]","[747.2111206054688, 1375.868408203125]","[747.1654663085938, 1376.5386962890625]","[747.1082763671875, 1377.1748046875]","[747.056884765625, 1377.8310546875]","[747.0004272460938, 1378.5111083984375]","[746.9462890625, 1379.2242431640625]","[746.8900756835938, 1379.9698486328125]","[746.8338623046875, 1380.7513427734375]",...,"[746.0184936523438, 1391.6556396484375]","[745.9192504882812, 1392.79150390625]","[745.8156127929688, 1393.9332275390625]","[745.707275390625, 1395.075439453125]","[745.59423828125, 1396.21142578125]","[745.4766845703125, 1397.334228515625]","[745.3550415039062, 1398.4366455078125]","[745.22998046875, 1399.5123291015625]","[745.102294921875, 1400.5565185546875]","[744.972412109375, 1401.565673828125]"
21235,"[746.1021118164062, 1386.4373779296875]","[745.4672241210938, 1385.956298828125]","[744.6143798828125, 1385.7894287109375]","[743.7474975585938, 1385.552978515625]","[742.9029541015625, 1385.3173828125]","[742.0640869140625, 1385.08544921875]","[741.2286376953125, 1384.8599853515625]","[740.3966674804688, 1384.6419677734375]","[739.5687255859375, 1384.4317626953125]","[738.745361328125, 1384.2293701171875]",...,"[729.9927368164062, 1382.501220703125]","[729.2109985351562, 1382.3804931640625]","[728.4280395507812, 1382.2642822265625]","[727.64306640625, 1382.15234375]","[726.8555297851562, 1382.044677734375]","[726.0650024414062, 1381.94140625]","[725.271240234375, 1381.

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
32150,580.654236,1382.880981,580.694763,1382.533813,580.633362,1382.161133,580.560608,1381.699463,580.503967,1381.291016,...,579.161194,1371.838379,579.129211,1371.345581,579.102478,1370.851074,579.081177,1370.355103,579.065491,1369.858032
6629,2147.945312,698.175354,2146.484375,696.827576,2145.405029,695.747437,2144.474121,694.816040,2143.590576,693.946655,...,2123.115723,675.439941,2122.250732,674.653137,2121.421875,673.881409,2120.629639,673.122498,2119.874756,672.374084
36283,414.653900,1526.989014,415.179260,1528.272095,415.210022,1529.187866,415.394684,1530.107300,415.525635,1531.066772,...,417.942993,1550.250366,418.004303,1551.145386,418.059540,1552.036865,418.108643,1552.922485,418.151672,1553.799316
29285,747.484375,1374.366455,747.301880,1375.154785,747.211121,1375.868408,747.165466,1376.538696,747.108276,1377.174805,...,745.476685,1397.334229,745.355042,1398.436646,745.229980,1399.512329,745.102295,1400.556519,744.972412,1401.565674
21235,746.102112,1386.437378,745.467224,1385.956299,744.614380,1385.789429,743.747498,1385.552979,742.902954,1385.317383,...,726.065002,1381.941406,725.271240,1381.842407,724.474182,1381.748169,723.673950,1381.658936,722.870911,1381.575439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15738,1788.596802,409.661652,1787.576416,408.938354,1786.786621,408.378632,1786.079346,407.815247,1785.409790,407.280792,...,1772.061401,396.666077,1771.450317,396.137268,1770.847290,395.608063,1770.253296,395.079163,1769.669800,394.551300
33327,1717.782104,350.865326,1716.802979,350.054230,1715.952881,349.317719,1715.145630,348.560974,1714.363770,347.831909,...,1698.124512,333.522461,1697.394409,332.858948,1696.679565,332.202271,1695.981445,331.553314,1695.301147,330.912750
10312,1778.284058,390.975647,1779.451294,391.893890,1780.759766,392.868744,1781.961182,393.754608,1783.118530,394.653412,...,1808.060791,414.213348,1809.012451,415.162201,1809.905273,416.087646,1810.740479,416.977173,1811.523315,417.822601


In [12]:
df2.to_csv("outputs5ep.csv", index=True)